# Dynamic Manning Report - Doctor Data

VADM Burke, Chief of Naval Personnel, ordered LTJGs Dannelly, Calcagno, Shaw, and Kuzma to build a prototype of a job-seeker / job-owner matching algorithm in one month to aid personnel detailing. This report offers an overview of the team’s results utilizing medical corps data provided by CDR Gingo. 

In [350]:
#Data Ingestion
#Import Libraries
import pandas as pd
import pref_metrics as pm

#Path to files
file_path_data = '/Users/z/Documents/Data_Cleaning/med_data/'
file_path_results = '/Users/z/Documents/Data_Cleaning/med_data/results/'

#Raw Data
s_df = pd.read_csv(file_path_data + "S.csv", index_col=0)
o_df = pd.read_csv(file_path_data + "O.csv", index_col=0)
a_df = pd.read_csv(file_path_data + "A.csv", index_col=0)

#Summary Stats

#Match Dataframes
x_MIP_df = pd.read_csv(file_path_results + "X_mip.csv", index_col=0)
x_da_s_df = pd.read_csv(file_path_results + "X_da_s.csv", index_col=0)
x_da_o_df = pd.read_csv(file_path_results + "X_da_o.csv", index_col=0)

#Similarity Dataframes
sim_s_df = pd.read_csv(file_path_results + "Similarity_s.csv", index_col=0)
sim_o_df = pd.read_csv(file_path_results + "Similarity_o.csv", index_col=0)

#Post Match Stats
post_stat_df = pd.read_json(file_path_results + "post_match.json")

#Metrics Dataframes
metrics_s_df = pd.read_csv(file_path_results + "Metrics_s.csv", index_col=0)
metrics_o_df = pd.read_csv(file_path_results + "Metrics_o.csv", index_col=0)

#Correlation Dataframes
corr_s_df = pd.read_csv(file_path_results + "Corr_s.csv", index_col=0)
corr_o_df = pd.read_csv(file_path_results + "Corr_o.csv", index_col=0)

#Get number of rankings for each comamnd
pref_counts_O = s_df.T.apply(pd.value_counts).drop(total_S).apply(pd.Series.value_counts)

#Get number of rankings for each sailor
pref_counts_S = o_df.T.apply(pd.value_counts).drop(total_O).apply(pd.Series.value_counts)
pref_counts_O = pref_counts_O.T
pref_counts_O

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,10.0,12.0,13.0
command_A,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
command_B,6.0,3.0,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
command_C,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
command_D,7.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
command_E,2.0,1.0,2.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
command_F,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
command_G,4.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
command_H,2.0,2.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,1.0
command_I,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
command_J,4.0,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [354]:
import plotly.plotly as py
import plotly.graph_objs as go

data = [
    go.Bar(
        x=pref_counts_O.index, # assign x as the dataframe column 'x'
        y=pref_counts_O[1.0],
        name = "Perf 1"
    ),
    go.Bar(
        x=pref_counts_O.index, # assign x as the dataframe column 'x'
        y=pref_counts_O[2.0],
        name = "Perf 2"
    ),
    go.Bar(
        x=pref_counts_O.index, # assign x as the dataframe column 'x'
        y=pref_counts_O[3.0],
        name = "Perf 3"
    ),
    go.Bar(
        x=pref_counts_O.index, # assign x as the dataframe column 'x'
        y=pref_counts_O[4.0],
        name = "Perf 4"
    )
    ,
    go.Bar(
        x=pref_counts_O.index, # assign x as the dataframe column 'x'
        y=pref_counts_O[5.0],
        name = "Perf 5"
    )
    ,
    go.Bar(
        x=pref_counts_O.index, # assign x as the dataframe column 'x'
        y=pref_counts_O[6.0],
        name = "Perf 6"
    )
    ,
    go.Bar(
        x=pref_counts_O.index, # assign x as the dataframe column 'x'
        y=pref_counts_O[7.0],
        name = "Perf 7"
    )
    ,
    go.Bar(
        x=pref_counts_O.index, # assign x as the dataframe column 'x'
        y=pref_counts_O[8.0],
        name = "Perf 8"
    )
    ,
    go.Bar(
        x=pref_counts_O.index, # assign x as the dataframe column 'x'
        y=pref_counts_O[10.0],
        name = "Perf 10"
    )

]

layout = go.Layout(
    barmode='stack',
    title='All e',
    yaxis=dict(
        tickformat='d'
    )
)

fig = go.Figure(data=data, layout=layout)

# IPython notebook
py.iplot(fig, filename='pandas-bar-chart-layout')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~dannellyz/0 or inside your plot.ly account where it is named 'pandas-bar-chart-layout'


## Preference Assignments

In [357]:
metrics_o_df

,Competitiveness,Specialization
command_A,0.6439,0.859231
command_B,0.7237,0.948132
command_C,0.6440,0.892597
command_D,0.6751,0.958260
command_E,0.7715,0.934041
command_F,0.6683,0.920081
command_G,0.6689,0.882353
command_H,0.8476,0.879592
command_I,0.6580,0.923127
command_J,0.6937,0.954807


In [254]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = {
  "data": [
    {
      "values": [16, 15, 12],
      "labels": [
        "first",
        "top5",
        "rest"
      ],
      'domain': {'x': [0, .5], 'y': [.52, 1]},
      "name": "1",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },
      {
      "values": [16, 15, 12],
      "labels": [
        "first",
        "top5",
        "rest"
      ],
      'domain': {'x': [0, .5], 'y': [0, .48]},
      "name": "2",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },
    {
      "values": [16, 15, 12],
      "labels": [
        "first",
        "top5",
        "rest"
      ],
      "text":["CO2"],
      "textposition":"inside",
      'domain': {'x': [.3, 1], 'y': [.1, .9]},
      "name": "3",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Global Emissions 1990-2011",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "DA-O",
                "x": 0.22,
                "y": 0.20
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "DA-S",
                "x": 0.22,
                "y": 0.80
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "MIP",
                "x": 0.7,
                "y": 0.5
            }
        ]
    }
}
py.iplot(fig, filename='donut')

In [268]:
metrics_s_df = (metrics_s_df-metrics_s_df.min())/(metrics_s_df.max()-metrics_s_df.min())
fig = {
    'data': [
      {
      'x': metrics_s_df.Competitiveness, 
            'y': metrics_s_df.Specialization, 
            'text': metrics_s_df.index, 
            'mode': 'markers', 
            'name': 'Sailors'}
    ],
    'layout': {
        'xaxis': {'title': 'Competitiveness', 'type': 'log'},
        'yaxis': {'title': "Specialization"}
    }
}

# IPython notebook
py.iplot(fig, filename='out')

,Competitiveness,Specialization
command_A,0.000000,0.000000
command_B,0.391753,0.897729
command_C,0.000491,0.336937
command_D,0.153166,1.000000
command_E,0.626411,0.755443
command_F,0.119784,0.614473
command_G,0.122730,0.233491
command_H,1.000000,0.205609
command_I,0.069219,0.645229
command_J,0.244477,0.965131


In [257]:
fig = {
    'data': [
      {
      'x': metrics_o_df.Competitiveness, 
            'y': metrics_o_df.Specialization, 
            'text': metrics_o_df.index, 
            'mode': 'markers', 
            'name': '2007'}
    ],
    'layout': {
        'xaxis': {'title': 'Competitiveness', 'type': 'log'},
        'yaxis': {'title': "Specialization"}
    }
}

# IPython notebook
py.iplot(fig, filename='out')

## Summary Data
This section offers statistical overviews of the sailors and job owners in the given data-set.

# Matching Algorithms

In [327]:
x_MIP_list = list(x_MIP_df[x_MIP_df > 0].stack().index)
x_da_o_list = list(x_da_o_df[x_da_o_df > 0].stack().index)
x_da_s_list = list(x_da_s_df[x_da_s_df > 0].stack().index)
da_s_dif_o = set(x_da_s_list) - set(x_da_o_list)
mip_dif_o = set(x_MIP_list) - set(x_da_o_list)
mip_dif_s = set(x_MIP_list) - set(x_da_s_list)

#print(x_da_o_list)
print(x_da_o_df.shape)
print(x_da_o_df.sum().sum())

print(x_da_s_df.shape)
print(x_da_s_df.sum().sum())
#print(x_da_s_list)

print(x_MIP_df.shape)
print(x_MIP_df.sum().sum())

(20, 59)
20
(20, 59)
57
(20, 59)
57


In [197]:
pref_counts = ((matched_results.T * O_df).astype(str) + '_' + (matched_results * S_df).T.astype(str)).stack().value_counts()
pref_df = pd.DataFrame(pref_counts).drop("0_0").reset_index()
goals = pref_df["index"].str.split("_", n = 1, expand = True)
labels = "JO-" + goals[0]
pref_viz_df = pd.DataFrame(columns=["Source","Target","Value","Color","Labels"])
pref_viz_df["Value"] = pref_df[0]
pref_viz_df["Labels"] = labels.append("S-" + goals[1], ignore_index=True).drop_duplicates().reset_index(drop=True)
pref_viz_df["Color"] = "#262C46"
pref_viz_df.fillna(0, inplace=True)
for index, row in goals.iterrows():
    pref_viz_df.at[index,"Source"] = pref_viz_df[pref_viz_df.Labels.str.contains("JO-" + row[0], na=False)].index.tolist()[0]
    pref_viz_df.at[index,"Target"] = pref_viz_df[pref_viz_df.Labels.str.contains("S-" + row[1], na=False)].index.tolist()[0]

pref_viz_df

,Source,Target,Value,Color,Labels
0,0,7,10,#262C46,JO-1
1,1,7,9,#262C46,JO-2
2,2,7,4,#262C46,JO-4
3,3,7,4,#262C46,JO-3
4,1,8,4,#262C46,JO-5
5,4,7,4,#262C46,JO-20
6,3,8,4,#262C46,JO-6
7,3,9,3,#262C46,S-1
8,5,9,2,#262C46,S-2
9,0,10,2,#262C46,S-3


In [200]:
import plotly.plotly as py

data_trace = dict(
    type='sankey',
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    valueformat = ".0f",
    node = dict(
      pad = 10,
      thickness = 30,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label =  pref_viz_df['Labels'].dropna(axis=0, how='any'),
      color = pref_viz_df['Color']
    ),
    link = dict(
      source = pref_viz_df['Source'].dropna(axis=0, how='any'),
      target = pref_viz_df['Target'].dropna(axis=0, how='any'),
      value = pref_viz_df['Value'].dropna(axis=0, how='any'),
  )
)

layout =  dict(
    title = "Results: Job Owner Preference to Sailor Preference",
    height = 772,
    width = 950,
    font = dict(
      size = 10
    ),    
)


fig = dict(data=[data_trace], layout=layout)
py.iplot(fig, validate=False)

In [217]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = {
  "data": [
    {
      "values": [16, 15, 12],
      "labels": [
        "first",
        "top5",
        "rest"
      ],
      'domain': {'x': [0, .5], 'y': [.52, 1]},
      "name": "1",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Global Emissions 1990-2011",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "DA-O",
                "x": 0.22,
                "y": 0.20
            }
        ]
    }
}
py.iplot(fig, filename='donut')

In [331]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

import pandas as pd
import numpy as np


N = 20
x = np.linspace(1, 10, N)
y = np.random.randn(N)+3
y2 = np.random.randn(N)+6
y3 = np.random.randn(N)+9
y4 = np.random.randn(N)+12
df = pd.DataFrame({'x': x, 'y': y, 'y2':y2, 'y3':y3, 'y4':y4})
df.head()

data = [
    go.Bar(
        x=df.x, # assign x as the dataframe column 'x'
        y=df.y1,
        name = "Perf 1"
    ),
    go.Bar(
        x=test.index,
        y=test['Pref 2'],
        name = "Perf 2"
    ),
    go.Bar(
        x=test.index,
        y=test['Pref 3'],
        name = "Perf 3"
    )

]

layout = go.Layout(
    barmode='stack',
    title='Stacked Bar with Pandas',
    yaxis=dict(
        tickformat='d'
    )
)

fig = go.Figure(data=data, layout=layout)

# IPython notebook
py.iplot(fig, filename='pandas-bar-chart-layout')

AttributeError: 'DataFrame' object has no attribute 'y1'

In [255]:
x_MIP_df

,officer_01,officer_02,officer_03,officer_04,officer_05,officer_06,officer_07,officer_08,officer_09,officer_10,...,officer_50,officer_51,officer_52,officer_53,officer_54,officer_55,officer_56,officer_57,officer_58,officer_59
command_A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
command_B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
command_C,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
command_D,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
command_E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
command_F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
command_G,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
command_H,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
command_I,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
command_J,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [284]:
O_comp = {}
for key, value in match_dict.items():
    total_comp = 0
    for item in value:
        total_comp += metrics_s_df.at[item,"Competitiveness"]
    O_comp[key] = total_comp
O_comp

{'command_A': 0.39634146341463183,
 'command_B': 0.17682926829268408,
 'command_C': 0.737804878048775,
 'command_D': 0.7926829268292704,
 'command_E': 2.0426829268292703,
 'command_F': 0.7926829268292704,
 'command_G': 0.7926829268292704,
 'command_H': 3.3292682926829316,
 'command_I': 0.4817073170731727,
 'command_J': 0.7134146341463387,
 'command_K': 1.0792682926829318,
 'command_L': 1.347560975609759,
 'command_M': 0.9390243902438955,
 'command_N': 1.0121951219512115,
 'command_O': 1.817073170731707,
 'command_P': 0.6951219512195114,
 'command_Q': 1.5365853658536546,
 'command_R': 0.7317073170731727,
 'command_S': 0.8963414634146318,
 'command_T': 1.6768292682926773}

In [290]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = {
  "data": [
    {
      "values": list(O_comp.values()),
      "labels": list(O_comp.keys()),
      "name": "1",
      "hoverinfo":"label+percent+name",
      "type": "pie"
    }],
  "layout": {
        "title":"Talent Allocation: Cumulative Competitiveness "
    }
}
py.iplot(fig, filename='donut')

In [288]:
list(O_comp.values)

<function dict.values>

In [291]:
sim_s_df

,officer_01,officer_02,officer_03,officer_04,officer_05,officer_06,officer_07,officer_08,officer_09,officer_10,...,officer_50,officer_51,officer_52,officer_53,officer_54,officer_55,officer_56,officer_57,officer_58,officer_59
officer_01,1.0000,-1.7081,-3.1114,-2.2201,-2.6168,-3.9333,-2.1081,-1.3585,0.0658,-2.7474,...,-2.9918,-1.8009,-1.3585,-3.2474,-1.8009,-2.4250,-0.8920,-2.2201,-1.8009,-1.5969
officer_02,-1.7081,1.0000,-2.3676,-1.2401,-1.6814,-3.5150,-2.1533,-1.2166,-1.7078,-1.8906,...,-2.0090,-1.6840,-2.1744,-2.2694,-1.6840,-1.0491,-1.7078,-2.0781,-1.6590,-1.5021
officer_03,-3.1114,-2.3676,1.0000,-1.9492,-3.2304,-3.0779,-1.8715,-1.9722,-3.0862,0.6222,...,-1.0004,-3.2474,-2.5949,-1.9924,-1.5021,-3.5624,-2.2739,-1.0555,-2.4146,-0.4870
officer_04,-2.2201,-1.2401,-1.9492,1.0000,-0.2818,-4.0426,-3.4109,0.1385,-1.3111,-1.6774,...,0.1590,-1.3114,-0.8696,-0.8716,0.5562,-3.1949,-0.3281,0.0632,-0.3040,-2.5975
officer_05,-2.6168,-1.6814,-3.2304,-0.2818,1.0000,-3.4637,-3.8076,-0.2582,-1.7078,-2.8231,...,-1.0765,0.1087,-2.1744,-1.3586,-0.7250,-3.6330,-0.7248,-1.1947,0.1606,-2.9942
officer_06,-3.9333,-3.5150,-3.0779,-4.0426,-3.4637,1.0000,-3.8743,-4.4334,-4.5255,-3.3306,...,-3.4892,-3.9281,-4.8394,-2.5622,-4.3413,-1.5431,-4.8275,-4.1064,-4.1059,-2.4974
officer_07,-2.1081,-2.1533,-1.8715,-3.4109,-3.8076,-3.8743,1.0000,-2.5494,-2.0829,-1.3746,...,-3.2499,-2.9918,-2.5494,-4.3264,-2.9918,-3.1288,-2.0829,-2.4279,-2.9918,-1.9143
officer_08,-1.3585,-1.2166,-1.9722,0.1385,-0.2582,-4.4334,-2.5494,1.0000,-0.4496,-1.5649,...,-0.6339,-0.4499,-0.9161,-1.6868,0.5332,-3.1032,0.5335,0.0635,0.5576,-1.7359
officer_09,0.0658,-1.7078,-3.0862,-1.3111,-1.7078,-4.5255,-2.0829,-0.4496,1.0000,-2.7713,...,-2.0829,-0.8920,-0.4496,-3.0862,-0.8920,-2.3995,0.0169,-1.3111,-0.8920,-2.2739
officer_10,-2.7474,-1.8906,0.6222,-1.6774,-2.8231,-3.3306,-1.3746,-1.5649,-2.7713,1.0000,...,-0.7633,-2.8652,-2.4256,-1.7732,-1.2385,-3.0606,-1.9355,-0.7384,-2.0073,-0.0004


In [ ]:
def to_dict(touples):

    from collections import defaultdict

    d1 = defaultdict(list)

    for k, v in touples:
        d1[k].append(v)

    return dict((k, tuple(v)) for k, v in d1.items())